In [24]:
import numpy as np 
import pandas as pd
import re  #resular expression for searching text in the document
from nltk.corpus import stopwords   #stopwords=unneccessory words
from nltk.stem.porter import PorterStemmer   #stemming function gives the root word in sentence
from sklearn.feature_extraction.text import TfidfVectorizer #to convert text into features vectors(numbers)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [25]:
import nltk
nltk.download('stopwords')   #downloading all stopwords

[nltk_data] Downloading package stopwords to C:\Users\Ayat laptop
[nltk_data]     point\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
print(stopwords.words('english'))   #these stop words does not add much value to the dataset

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
print("Languages with stopwords in NLTK:")
languages = stopwords.fileids()
for language in languages:
    print(language)

Languages with stopwords in NLTK:
arabic
azerbaijani
basque
bengali
catalan
chinese
danish
dutch
english
finnish
french
german
greek
hebrew
hinglish
hungarian
indonesian
italian
kazakh
nepali
norwegian
portuguese
romanian
russian
slovene
spanish
swedish
tajik
turkish


In [28]:
news = pd.read_csv('train.csv')

In [29]:
news

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [30]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [31]:
# Calculate the percentage of missing values for each column
missing_percentage = news.isnull().mean() * 100

# Display the result
print(missing_percentage)

id        0.000000
title     2.682692
author    9.408654
text      0.187500
label     0.000000
dtype: float64


In [32]:
#replacing the null values with empty string
news = news.fillna('')

In [33]:
news.isnull().sum()  #checking again

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [34]:
#combinng title and author column together
news['content'] = news['author']+' '+news['title']

In [35]:
print(news['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [36]:
def stemming(content):
    # Create a Porter Stemmer object
    porter = PorterStemmer()
    
    # Remove numbers and punctuation
    content = re.sub(r'[^\w\s]', ' ', content)   # r'[^\w\s] means exclude everything that does not include in a-z & A-Z set, eg: nuber, punctutation etc   from the content columns which we had made by combining author and title columns.
    
    # Convert to lowercase
    content = content.lower()
    
    # Split into words
    content = content.split()
    
    # Remove stopwords and apply stemming
    stop_words = set(stopwords.words('english'))
    content = [porter.stem(word) for word in content if word not in stop_words]  #using a for loop to choose all the words that are not the stop words and performing the functiion porterstemmer by using variable port
    
    # Join words back into a single string
    stemmed_content = ' '.join(content)
    
    return stemmed_content

In [37]:
news['content'] = news['content'].apply(stemming)

In [38]:
print(news['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                         david swanson keep f 35 aliv
Name: content, Length: 20800, dtype: object


In [39]:
#we are not goin to use the text column, because it is way too long , instd of this we are going to use the tital an author column
#making x and y again , only using the content(title+author) for the x
x=news['content'].values
y=news['label'].values

In [40]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f 35 aliv']


In [41]:
print(y)

[1 0 1 ... 0 1 1]


In [45]:
#using the TfidfVectorizer function to convert the text into numbers
vect = TfidfVectorizer()
vect.fit(x)  #taking all the words in x for conversion

x=vect.transform(x)  #converting the words taken into feature vectors(numbers)

In [46]:
print(x)

  (0, 16255)	0.28485063562728646
  (0, 14053)	0.2565896679337957
  (0, 9448)	0.3635963806326075
  (0, 9179)	0.29212514087043684
  (0, 8240)	0.24785219520671603
  (0, 7543)	0.21874169089359144
  (0, 5489)	0.233316966909351
  (0, 4285)	0.2705332480845492
  (0, 4093)	0.3598939188262559
  (0, 3446)	0.2468450128533713
  (0, 2976)	0.3676519686797209
  (0, 740)	0.27010124977708766
  (1, 17375)	0.30071745655510157
  (1, 7355)	0.1904660198296849
  (1, 6023)	0.7143299355715573
  (1, 4061)	0.26373768806048464
  (1, 3298)	0.19094574062359204
  (1, 2712)	0.3827320386859759
  (1, 2376)	0.15521974226349364
  (1, 1981)	0.2939891562094648
  (2, 16179)	0.41544962664721613
  (2, 10175)	0.49351492943649944
  (2, 6505)	0.3474613386728292
  (2, 5909)	0.3866530551182615
  (2, 3592)	0.46097489583229645
  :	:
  (20797, 12925)	0.27263457663336677
  (20797, 12720)	0.24778257724396507
  (20797, 10866)	0.08038079000566466
  (20797, 10143)	0.174553480255222
  (20797, 10073)	0.2954204003420313
  (20797, 9538)	0.3616

In [47]:
x.shape

(20800, 18741)

In [50]:
#using train_test_split here
xtr,xts,ytr,yts = train_test_split(x,y,test_size=0.2,stratify=y, random_state=2)  #changing random_state vale will split hte data in differe nt differnet manners

In [53]:
#training LR model
lr = LogisticRegression()   #LR sets a threshold value eg:0.5 , if value exceeds than Yes, if Below than 0.5 than No.

#  x--input features     
#  y--prediction probability    0/1
#  w--weights      importance of features/column
#  b--biases      the intercept

In [57]:
lr.fit(xtr,ytr)  #training the model with xtrain and y train

LogisticRegression()

In [61]:
#finding accuracy score  taring
xtr_pred = lr.predict(xtr)
xtr_accuracy = accuracy_score(xtr_pred,ytr)   # xtr_pred aare the values prdicted by our model and ytr is the original labels
print("accuracy score of training data:" ,xtr_accuracy)

accuracy score of training data: 0.9866586538461538


In [63]:
#finding accuracy score  testing
xts_pred = lr.predict(xts)
xts_accuracy = accuracy_score(xts_pred,yts)   # xts_pred aare the values prdicted by our model and yts is the original labels
print("accuracy score of test data:" ,xts_accuracy)

accuracy score of test data: 0.9788461538461538


In [72]:
#predictive system
x_new = xts[65]   #taking the 1st row in xtest data and storing it in a new varable

prediction = lr.predict(x_new)
print(prediction)

if (prediction[0]==0):   #0 means real news
    print('the news is real')
else:
    print('the news is fake')

[1]
the news is fake


In [73]:
#doublechecing
print(yts[65])

1
